In [1]:
!pip install discord.py
!pip3 install discord.py
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.8 MB/s eta 0:00:00


In [ ]:
import discord
from discord.ext import commands

class CommandNode:
    def __init__(self, command, prev=None, next=None):
        self.command = command
        self.prev = prev
        self.next = next

class CommandHistory:
    def __init__(self):
        self.current = None
        self.head = None

    def add_command(self, command):
        new_node = CommandNode(command, prev=self.current)
        if self.current:
            self.current.next = new_node
        self.current = new_node
        if not self.head:
            self.head = new_node

    def get_last_command(self):
        return self.current.command if self.current else None

    def get_user_commands(self):
        user_commands = []
        current_node = self.head
        while current_node:
            user_commands.append(current_node.command)
            current_node = current_node.next
        return user_commands

    def move_forward(self):
        if self.current and self.current.next:
            self.current = self.current.next
            return self.current.command
        return None

    def move_backward(self):
        if self.current and self.current.prev:
            self.current = self.current.prev
            return self.current.command
        return None

    def clear_history(self):
        self.head = None
        self.current = None

intents = discord.Intents.all()
client = commands.Bot(command_prefix="!", intents=intents)

history = CommandHistory()

@client.command(name="Hello")
async def hello(ctx):
    messages = await ctx.channel.history(limit=10).flatten()

    for each_message in messages:
        await each_message.delete()

@client.command(name="AddCommand")
async def add_command(ctx, command):
    history.add_command(command)
    await ctx.send(f"Commande ajoutée à l'historique : {command}")

@client.command(name="ShowLastCommand")
async def show_last_command(ctx):
    last_command = history.get_last_command()
    await ctx.send(f"Dernière commande : {last_command}")

@client.command(name="ShowUserCommands")
async def show_user_commands(ctx):
    user_commands = history.get_user_commands()
    await ctx.send(f"Historique de l'utilisateur : {user_commands}")

@client.command(name="MoveForward")
async def move_forward(ctx):
    result = history.move_forward()
    if result is not None:
        await ctx.send(f"Commande suivante : {result}")
    else:
        await ctx.send("Aucune commande suivante dans l'historique")

@client.command(name="MoveBackward")
async def move_backward(ctx):
    result = history.move_backward()
    if result is not None:
        await ctx.send(f"Commande précédente : {result}")
    else:
        await ctx.send("Aucune commande précédente dans l'historique")

@client.command(name="ClearHistory")
async def clear_history(ctx):
    history.clear_history()
    await ctx.send("Historique vidé")

@client.event
async def on_ready():
    print("Le bot est prêt !")


client.run("MTE2NzQ2OTMxOTIwMzA3ODE1NA.GANbWu.JXpFE5FjLqzD-duC2i_VX65BHaAFoWNjo9P7U4")